In [1]:
cd /Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/P3HT

/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/P3HT


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [3]:
dataset = pd.read_csv(r'/Users/Liangqiaohao/Desktop/PV Lab/2020/March/Learning project/P3HT/P3HT_dataset.csv')
X = dataset.iloc[:,5:10]
# i put negative sign here because default BO is for minimization
X['Conductivity'] = -dataset.iloc[:,13].values
X


,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
0,45.00,0.00,55.00,0.00,0.00,-12.77
1,75.00,0.00,25.00,0.00,0.00,-13.19
2,30.00,0.00,70.00,0.00,0.00,-14.78
3,30.00,0.00,70.00,0.00,0.00,-16.34
4,45.00,0.00,55.00,0.00,0.00,-16.94
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,-697.40
229,44.30,54.68,0.04,0.29,0.78,-731.11
230,38.92,49.28,0.09,0.95,10.67,-738.10
231,44.30,54.68,0.04,0.29,0.78,-772.94


In [4]:
X_run = X.groupby(['P3HT content (%)', 'D1 content (%)','D2 content (%)','D6 content (%)','D8 content (%)'])['Conductivity'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()
X_run

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity
0,15.00,0.00,0.00,85.00,0.00,-45.970
1,16.15,0.46,25.38,51.42,6.55,-16.905
2,16.88,1.25,24.96,49.58,7.32,-20.030
3,17.71,0.98,25.34,50.30,5.68,-14.780
4,18.67,0.27,47.09,24.03,9.99,-14.080
...,...,...,...,...,...,...
173,93.87,0.24,0.56,0.24,5.11,-9.020
174,94.58,0.03,1.15,1.38,2.91,-9.710
175,94.81,0.30,0.28,0.08,4.53,-6.440
176,95.00,0.00,0.00,0.00,5.00,-26.190


In [5]:
features = ['P3HT content (%)', 'D1 content (%)','D2 content (%)','D6 content (%)','D8 content (%)']
X_feature = X_run[['P3HT content (%)', 'D1 content (%)','D2 content (%)','D6 content (%)','D8 content (%)']].values

X_Conductivity = np.array(X_run['Conductivity'].values)


In [6]:
bounds = [{'name':'P3HT content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D1 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D2 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D6 content (%)','type':'continuous','domain':(0, 100)},
          {'name':'D8 content (%)','type':'continuous','domain':(0, 100)}
         ]

constraints_baseline = [{'name': 'constr_1', 'constraint': 'x[:,0] + x[:,1] + x[:,2] + x[:,3] + x[:,4]- 100'}
                       ]

# constraints_baseline = [{'name': 'constr_1', 'constraint': 'x[:,0] + x[:,1] + x[:,2] + x[:,3] + x[:,4]- 100'},
#                         {'name': 'constr_2', 'constraint': '100.1 - x[:,0] - x[:,1] - x[:,2] - x[:,3] - x[:,4]'}
#                        ]


In [7]:
# total 178 pts
n_initial = 10
n_test = int(np.round(len(X_Conductivity) * 0.1))
jitter = 0.01

top10_indices = list(X_run.sort_values('Conductivity').head(10).index)

seed_list = [74,660,861,1526,2,151,19,7,33,4021,66,42424,671,25,95711,850,71,2148,26219,4]
# 20

n_estimators = 50

In [8]:


def EI(X, model, y_best, input_mean, input_std):
#     xi = 0.01
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
#     z = (mean - y_best - xi)/std   "this is for maximization"
#     return (mean - y_best - xi) * norm.cdf(z) + std * norm.pdf(z)
#     z = (y_best - mean + xi)/std
#     return (y_best - mean + xi) * norm.cdf(z) + std * norm.pdf(z)
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, model, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    return 2 * std - mean

def MPI(X, model, y_best, input_mean, input_std):
    normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    mean = normalized_mean * input_std + input_mean
    std = normalized_std * input_std
    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, model, input_mean, input_std):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_Conductivity[index]
        normalized_mean, normalized_std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
        mean = normalized_mean * input_std + input_mean
        std = normalized_std * input_std
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

def Top10Percent(top10_index, index_):
    how_many = 0
    for i in index_:
        if i in top10_index:
            how_many += 1
    return how_many
 


In [33]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
#         print('started '+str(i) + ' round')
        
        input_mean = np.mean(Conductivity_)
        input_std = np.std(Conductivity_)
        input_best = np.min(Conductivity_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Conductivity_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            ac_value = EI(X_j, BO_model.model, input_best, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
#         print(next_index)
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(i) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('BO_EI_master', master)
   

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [34]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(Conductivity_)
        input_std = np.std(Conductivity_)
        input_best = np.min(Conductivity_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'LCB',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Conductivity_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            
            ac_value = LCB(X_j, BO_model.model, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
  
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('BO_LCB_master', master)

initializing seed = 0
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The set cost function is ignored! LCB acquisition does not make sense with cost.
The se

In [35]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_mean = np.mean(Conductivity_)
        input_std = np.std(Conductivity_)
        input_best = np.min(Conductivity_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'MPI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Conductivity_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            
            ac_value = MPI(X_j, BO_model.model, input_best, input_mean, input_std)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('BO_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [36]:
# random sampling

index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []

for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial

    for i in np.arange(n_opt):
    
        input_mean = np.mean(Conductivity_)
        input_std = np.std(Conductivity_)
        input_best = np.min(Conductivity_)
        
        BO_model = GPyOpt.methods.BayesianOptimization(f=None,  
                                                   domain = bounds,
                                                   constraints = constraints_baseline,
                                                   acquisition_type = 'EI',              
                                                   normalize_Y = True,
                                                   X = np.array(X_),
                                                   Y = np.array([[i] for i in Conductivity_]),
                                                   evaluator_type = 'local_penalization',
                                                   batch_size = 1,
                                                   acquisition_jitter = jitter)

        _ = BO_model.suggest_next_locations()
        
        next_index = (random.sample(indices, 1))[0]
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, BO_model.model, input_mean, input_std)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('RS_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [10]:
def EI(X, RF_model, y_best):

    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 2 * std - mean

def MPI(X, RF_model, y_best):
    
    tree_predictions = []
    for j in np.arange(n_estimators):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]
    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, RF_model):
    total_mae = 0
    for index in test_index:
        X = X_feature[index]
        y = X_Conductivity[index]
        
        tree_predictions = []
        for j in np.arange(n_estimators):
            tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
        mean = np.mean(np.array(tree_predictions), axis=0)[0]

        std = np.std(np.array(tree_predictions), axis=0)[0]
        
        mae_ = (y - mean)**2
        total_mae += mae_
    return total_mae / len(test_index)

In [11]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Conductivity_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Conductivity_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            
            ac_value = EI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('RF_EI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [13]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Conductivity_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Conductivity_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('RF_LCB_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed


In [14]:
index_collection = []
X_collection = []
Conductivity_collection = []
InTopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    indices = list(np.arange(len(X_run)))
    
    top10_index = top10_indices
    X_top10 = []
    Conductivity_top10 = []
    for i in top10_index:
        X_top10.append(X_feature[i])
        Conductivity_top10.append(X_Conductivity[i])
    
    index_ = random.sample(indices, n_initial)
    X_ = []
    Conductivity_ = []
    InTopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        Conductivity_.append(X_Conductivity[i])
        indices.remove(i)
    
    test_index = random.sample(indices, n_test)
    X_test = []
    Conductivity_test = []
    for i in test_index:
        X_test.append(X_feature[i])
        Conductivity_test.append(X_Conductivity[i])
        indices.remove(i)
    
    n_opt = len(indices)
    assert n_opt == len(X_run) - n_test - n_initial
    
    for i in np.arange(n_opt):
        
        input_best = np.min(Conductivity_)
        
        RF_model = RandomForestRegressor(n_estimators=50)
        RF_model.fit(X_, Conductivity_)
        
        
        next_index = None
        max_ac = -10**10
        for j in indices:
            X_j = X_feature[j]
            Conductivity_j = X_Conductivity[j]
            
            ac_value = MPI(X_j, RF_model, input_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        Conductivity_.append(X_Conductivity[next_index])
        indices.remove(next_index)
        index_.append(next_index)
        mae = calc_MAE(test_index, RF_model)
        MAE_.append(mae)
        how_many = Top10Percent(top10_index, index_)
        InTopPercent_.append(how_many)
        
#         print('Finished '+str(len(MAE_)) + ' round')
        
    assert len(MAE_) == len(X_) - n_initial

    index_collection.append(index_)
    X_collection.append(X_)
    Conductivity_collection.append(Conductivity_)
    InTopPercent_collection.append(InTopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, Conductivity_collection, InTopPercent_collection, MAE_collection])
np.save('RF_MPI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
